In [1]:
import pandas as pd


Import and clean GBG data

In [2]:
df_gbg = pd.read_csv('../Data/smhi_temp_data_gbg.csv', usecols=[0, 1, 2, 3], header=11, sep=';')

In [3]:
df_gbg['Snittemperatur'] = df_gbg.groupby('Datum')['Lufttemperatur'].transform('mean').round(2)
df_gbg = df_gbg.groupby('Datum').max().drop('Lufttemperatur', axis=1).reset_index()

In [4]:
df_gbg['Datum'] = pd.to_datetime(df_gbg['Datum'])
df_gbg['Year'] = df_gbg['Datum'].dt.year
df_gbg['Month'] = df_gbg['Datum'].dt.month
df_gbg = df_gbg.drop(columns=['Tid (UTC)'])

Import and clean Säve data

In [5]:
df_save = pd.read_csv('../Data/smhi_temp_data_save.csv', usecols=[0, 1, 2, 3], header=6, sep=';')

In [6]:
df_save['Snittemperatur'] = df_save.groupby('Datum')['Lufttemperatur'].transform('mean').round(2)
df_save = df_save.groupby('Datum').max().drop('Lufttemperatur', axis=1).reset_index()

In [7]:
df_save['Datum'] = pd.to_datetime(df_save['Datum'])
df_save['Year'] = df_save['Datum'].dt.year
df_save['Month'] = df_save['Datum'].dt.month
df_save = df_save.drop(columns=['Tid (UTC)'])

In [8]:
df_save

,Datum,Kvalitet,Snittemperatur,Year,Month
0,1944-01-01,G,3.20,1944,1
1,1944-01-02,G,3.60,1944,1
2,1944-01-03,G,2.80,1944,1
3,1944-01-04,G,-4.47,1944,1
4,1944-01-05,G,-1.40,1944,1
...,...,...,...,...,...
22980,2006-12-01,Y,8.03,2006,12
22981,2006-12-02,Y,8.81,2006,12
22982,2006-12-03,Y,8.11,2006,12
22983,2006-12-04,Y,8.57,2006,12


In [9]:
df_gbg

,Datum,Kvalitet,Snittemperatur,Year,Month
0,1961-01-01,G,1.07,1961,1
1,1961-01-02,G,2.27,1961,1
2,1961-01-03,G,2.27,1961,1
3,1961-01-04,G,2.53,1961,1
4,1961-01-05,G,2.13,1961,1
...,...,...,...,...,...
19909,2023-06-27,G,18.18,2023,6
19910,2023-06-28,G,19.72,2023,6
19911,2023-06-29,G,19.22,2023,6
19912,2023-06-30,G,17.40,2023,6


Save both datasets as csv and pkl

In [10]:
# df_gbg.to_csv('../data/prepared_data/daily_temp_gbg.csv', index=False, mode='w')
# df_save.to_csv('../data/prepared_data/daily_temp_save.csv', index=False, mode='w')

In [11]:
# df_gbg.to_pickle('../Dataframes/df_daily_temp_gbg.pkl')
# df_save.to_pickle('../Dataframes/df_daily_temp_save.pkl')

Combine borh GBG and Säve
Let Säve data run all the way until last day, let GBG take over

In [12]:
# Last fay of Säve data was not a complete day, remove it.
df_save.drop(df_save.tail(1).index, inplace=True)


In [13]:
df_save

,Datum,Kvalitet,Snittemperatur,Year,Month
0,1944-01-01,G,3.20,1944,1
1,1944-01-02,G,3.60,1944,1
2,1944-01-03,G,2.80,1944,1
3,1944-01-04,G,-4.47,1944,1
4,1944-01-05,G,-1.40,1944,1
...,...,...,...,...,...
22979,2006-11-30,Y,9.98,2006,11
22980,2006-12-01,Y,8.03,2006,12
22981,2006-12-02,Y,8.81,2006,12
22982,2006-12-03,Y,8.11,2006,12


In [14]:
last_day_save = df_save['Datum'][-1:]

In [15]:
last_day_save = str(last_day_save.values[0])[0:10]

In [16]:
last_day_save

'2006-12-04'

In [17]:
df_gbg_compile = df_gbg[df_gbg['Datum'] > last_day_save]

In [18]:
df_compiled = pd.concat([df_save, df_gbg_compile])

In [19]:
df_compiled

,Datum,Kvalitet,Snittemperatur,Year,Month
0,1944-01-01,G,3.20,1944,1
1,1944-01-02,G,3.60,1944,1
2,1944-01-03,G,2.80,1944,1
3,1944-01-04,G,-4.47,1944,1
4,1944-01-05,G,-1.40,1944,1
...,...,...,...,...,...
19909,2023-06-27,G,18.18,2023,6
19910,2023-06-28,G,19.72,2023,6
19911,2023-06-29,G,19.22,2023,6
19912,2023-06-30,G,17.40,2023,6


In [20]:
# df_compiled.to_csv('../data/prepared_data/compiled_daily_temp_gbg_save.csv', index=False)

In [21]:
# df_compiled.to_pickle('../Dataframes/df_compiled_daily_temp_gbg_save.pkl')

Make monthly temp data

In [22]:
df_gbg_monthly_temp = df_gbg.groupby(['Year', 'Month'])['Snittemperatur'].mean().round(2).reset_index()

In [23]:
df_save_monthly_temp = df_save.groupby(['Year', 'Month'])['Snittemperatur'].mean().round(2).reset_index()

In [24]:
# df_gbg_monthly_temp.to_csv('../data/prepared_data/monthly_temp_gbg.csv', index=False)
# df_save_monthly_temp.to_csv('../data/prepared_data/monthly_temp_save.csv', index=False)

In [25]:
# df_gbg_monthly_temp.to_pickle('../Dataframes/df_monthly_temp_gbg.pkl')
# df_save_monthly_temp.to_pickle('../Dataframes/df_monthly_temp_save.pkl')

Combine the two

In [26]:
df_compiled_monthly_temp = pd.concat([df_save_monthly_temp, df_gbg_monthly_temp])

In [28]:
# df_compiled_monthly_temp.to_csv('../data/prepared_data/compiled_monthly_temp_gbg_save.csv', index=False, mode='w')
# df_compiled_monthly_temp.to_pickle('../Dataframes/df_compiled_monthly_temp_gbg_save.pkl')